In [1]:
import pickle

file_get = open("../valid_t_dict.pkl",'rb') 
valid_t_dict = pickle.load(file_get) 
file_get.close()

In [2]:
path = "D:/share/TOSS/old"

In [3]:
file_get= open(path + "/global_normalized_normed_dict_loop_0.pkl","rb")
global_normalized_normed_dict = pickle.load(file_get)
file_get.close()

file_get= open(path + "/global_mean_dict_loop_0.pkl","rb")
global_mean_dict = pickle.load(file_get)
file_get.close()

file_get= open(path + "/global_sigma_dict_loop_0.pkl","rb")
global_sigma_dict = pickle.load(file_get)
file_get.close()

In [41]:
from post_process import *
import copy
import itertools
import math
import numpy as np
import random

class TUNE():
    def __init__(self):
        None

    def apply_resonance(self, atom_idx_list, valence_list, LOSS, likelyhood, prior, res, global_nomalized_dict, 
                                                                                         global_sigma_dict, 
                                                                                         global_mean_dict):
        ori_LOSS = copy.deepcopy(LOSS)
        ori_LIKELYHOOD = copy.deepcopy(likelyhood)
        ori_PRIOR = copy.deepcopy(prior)

        classes_dict = classify(atom_idx_list,res)
        valence_with_loss = []
        NC = len(classes_dict)
        for c in classes_dict:
            for d in classes_dict:
                if c[0] != d[0]:
                    op_c = classes_dict[c]
                    op_d = classes_dict[d]

                    now_valence_list_c = [valence_list[i] for i in op_c]
                    now_max_oxi_list_c = [res.max_oxi_list[i] for i in op_c]
                    now_min_oxi_list_c = [max(0,res.min_oxi_list[i]) for i in op_c]   #because they are the super atom!
                    now_valence_list_d = [valence_list[i] for i in op_d]
                    now_max_oxi_list_d = [res.max_oxi_list[i] for i in op_d]
                    now_min_oxi_list_d = [max(0,res.min_oxi_list[i]) for i in op_d]   #same reason.

                    if len(op_c) < len(op_d):
                        now_valence_list_c = [i + 1 for i in now_valence_list_c]
                        N = len(op_c)
                        while N != 0:
                            temp_list = sorted([(j, v) for j,v in enumerate(now_valence_list_d)], reverse = True, key = lambda x:x[1])
                            now_valence_list_d[temp_list[0][0]] -= 1
                            N -= 1
                            #print("38", N, res.mid)

                        target_valence = set(now_valence_list_d)

                        #make sure the volume of the memory!
                        total_combi = math.factorial(len(op_d))/math.factorial(len(op_d)-len(op_c))/math.factorial(len(op_c))
                        while total_combi >= 10000000:
                            op_d = random.sample(op_d, round(len(op_d)*(4/5)))
                            total_combi = math.factorial(len(op_d))/math.factorial(len(op_d)-len(op_c))/math.factorial(len(op_c))
                            #print("47", total_combi, res.mid)

                        possible_combi = []
                        for p in itertools.combinations(op_d, len(op_c)):
                            possible_combi.append(p)

                        num_sample = round(3000/(NC**2)) if int(res.resonance_order) < 2 else round((3000)**(1/2)/(NC**2))
                        if len(possible_combi) >= num_sample:
                            possible_combi = random.sample(possible_combi, num_sample)

                        resonance = {"op_d": possible_combi, "target_valence":target_valence}
                        

                    if len(op_c) > len(op_d):
                        now_valence_list_d = [j - 1 for j in now_valence_list_d]
                        N = len(op_d)
                        while N != 0:
                            temp_list = sorted([(i, v) for i,v in enumerate(now_valence_list_c)], reverse = False, key = lambda x:x[1])
                            now_valence_list_c[temp_list[0][0]] += 1
                            N -= 1
                            #print("65", N, res.mid)
                        
                        target_valence = set(now_valence_list_c)

                        #make sure the volume of the memory!
                        total_combi = math.factorial(len(op_c))/math.factorial(len(op_c)-len(op_d))/math.factorial(len(op_d))
                        while total_combi >= 10000000:
                            op_c = random.sample(op_c, round(len(op_c)*(4/5)))
                            total_combi = math.factorial(len(op_c))/math.factorial(len(op_c)-len(op_d))/math.factorial(len(op_d))
                            #print("74", total_combi, res.mid)
                        possible_combi = []

                        for p in itertools.combinations(op_c, len(op_d)):
                            possible_combi.append(p)

                        num_sample = round(3000/(NC**2)) if int(res.resonance_order) < 2 else round((3000)**(1/2)/(NC**2))
                        if len(possible_combi) >= num_sample:
                            possible_combi = random.sample(possible_combi, num_sample)

                        resonance = {"op_c": possible_combi,"target_valence":target_valence}
                        
                
                    if len(op_c) == len(op_d):
                        now_valence_list_c = [i + 1 for i in now_valence_list_c]
                        now_valence_list_d = [j - 1 for j in now_valence_list_d]
                        resonance = {"equal": [tuple(op_c)]}

                    if all([now_max_oxi_list_c[i] >= now_valence_list_c[i] for i in range(len(op_c))]) and all(
                            [now_min_oxi_list_d[i] <= now_valence_list_d[i] for i in range(len(op_d))]):
                        
                        resonance_result = []
                        if list(resonance.keys())[0] == "op_d":
                            possible_valence_list = []
                            for p in resonance["op_d"]:
                                temp_sum_of_valence = copy.deepcopy(valence_list)
                                for i in op_c:
                                    temp_sum_of_valence[i] += 1
                                for j in p:
                                    temp_sum_of_valence[j] -= 1
                                if set([temp_sum_of_valence[i] for i in op_d]) == resonance["target_valence"]:
                                    possible_valence_list.append(temp_sum_of_valence)                                         

                        if list(resonance.keys())[0] == "op_c":
                            possible_valence_list = []
                            for p in resonance["op_c"]:
                                temp_sum_of_valence = copy.deepcopy(valence_list)
                                for i in p:
                                    temp_sum_of_valence[i] += 1
                                for j in op_d:
                                    temp_sum_of_valence[j] -= 1
                                if set([temp_sum_of_valence[i] for i in op_c]) == resonance["target_valence"]:
                                    possible_valence_list.append(temp_sum_of_valence)

                        if list(resonance.keys())[0] == "equal":
                            temp_sum_of_valence = copy.deepcopy(valence_list)
                            for i in op_c:
                                temp_sum_of_valence[i] += 1
                            for j in op_d:
                                temp_sum_of_valence[j] -= 1
                            possible_valence_list = [temp_sum_of_valence] 

                        for vl in possible_valence_list:
                            temp_pair_info = spider_pair_length_with_CN_unnorm(vl, res)
                            temp_LOSS, temp_LIKELYHOOD, temp_PRIOR = cal_loss_func_by_MAP(temp_pair_info,global_nomalized_dict, global_sigma_dict, global_mean_dict)
                            valence_with_loss.append((vl,temp_LOSS,temp_LIKELYHOOD, temp_PRIOR))

                    else:
                        valence_with_loss.append((valence_list, ori_LOSS, ori_LIKELYHOOD, ori_PRIOR))

        return valence_with_loss


    def tune_by_resonance(self, LOSS, likelyhood, prior, res, global_nomalized_dict, global_sigma_dict, global_mean_dict):
        valence_with_loss = self.apply_resonance(res.super_atom_idx_list, 
                                                 res.sum_of_valence, 
                                                 LOSS, 
                                                 likelyhood,
                                                 prior,
                                                 res,
                                                 global_nomalized_dict, 
                                                 global_sigma_dict, 
                                                 global_mean_dict)  #first run
        #print(valence_with_loss)
        check = {}
        #print(valence_with_loss)
        for vwl in valence_with_loss:
            if str(sorted(vwl[0])) not in check:
                check[str(sorted(vwl[0]))] = [vwl]
            else:
                if vwl not in check[str(sorted(vwl[0]))]:
                    check[str(sorted(vwl[0]))].append(vwl)

        if res.resonance_order == "2":
            for vwl in valence_with_loss:
                prime_valence_with_loss = self.apply_resonance(res.super_atom_idx_list, 
                                                               vwl[0], 
                                                               vwl[1], 
                                                               vwl[2],
                                                               vwl[3],
                                                               res, 
                                                               global_nomalized_dict, 
                                                               global_sigma_dict, 
                                                               global_mean_dict)
                for p in prime_valence_with_loss:
                    if str(sorted(p[0])) not in check:
                        check[str(sorted(p[0]))] = [p]
                    else:
                        if p not in check[str(sorted(p[0]))]:
                            check[str(sorted(p[0]))].append(p)

        possible_resonance = {}
        for k,v in check.items():
            avg_loss = sum(l[1] for l in v)/len(v)
            possible_resonance[avg_loss] = v

        key = sorted(possible_resonance.keys())[0]

        the_resonance_result = possible_resonance[key]
        avg_LOSS = sum([i[1] for i in the_resonance_result])/len(the_resonance_result)
        #one_sum_of_valence = the_resonance_result[0][0]
        resonance_valence_list = [i[0] for i in the_resonance_result]
        return avg_LOSS, the_resonance_result


###########################################################################################################
###################################################OLS APPROACH############################################
###########################################################################################################
    def apply_redox_by_OLS(self, classes_dict, valence_list, res,global_nomalized_dict):
        loss_matrix_dict = {}
        for c in classes_dict: #LOSE electron, valence plus 1
            for d in classes_dict: #get electron, valence minus 1
                if c[0] != d[0]:
                    now_valence_list_i = [valence_list[i] for i in classes_dict[c]]
                    now_max_oxi_list_i = [res.max_oxi_list[i] for i in classes_dict[c]]
                    now_min_oxi_list_i = [max(1,res.min_oxi_list[i]) for i in classes_dict[c]]  #because the center atom must > 0, right!?
                    now_valence_list_j = [valence_list[i] for i in classes_dict[d]]
                    now_max_oxi_list_j = [res.max_oxi_list[i] for i in classes_dict[d]]
                    now_min_oxi_list_j = [max(1,res.min_oxi_list[i]) for i in classes_dict[d]]  #because the center atom must > 0, right!?

                    num_c = len(classes_dict[c])
                    num_d = len(classes_dict[d])
                    lcm = num_c*num_d/math.gcd(num_c,num_d)
                    num_lose = int(lcm/num_c)
                    num_get = int(lcm/num_d)
                    now_valence_list_i = [i + num_lose for i in now_valence_list_i]
                    now_valence_list_j = [i - num_get for i in now_valence_list_j]

                    if all([now_max_oxi_list_i[i] >= now_valence_list_i[i] for i in range(len(classes_dict[c]))]) and all([now_min_oxi_list_j[i] <= now_valence_list_j[i] for i in range(len(classes_dict[d]))]):
                        for i in classes_dict[c]:
                            valence_list[i] += num_lose
                        for i in classes_dict[d]:
                            valence_list[i] -= num_get

                        try:
                            temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                            LOSS = cal_loss_func_by_OLS(temp_pair_info,global_nomalized_dict)

                        except:
                            LOSS = 100000
                        loss_matrix_dict[LOSS] = copy.deepcopy(valence_list)
                        for i in classes_dict[c]:
                            valence_list[i] -= num_lose
                        for i in classes_dict[d]:
                            valence_list[i] += num_get
                    else:
                        temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                        LOSS = cal_loss_func_by_OLS(temp_pair_info,global_nomalized_dict)
                        loss_matrix_dict[LOSS] = valence_list
                else:
                    temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                    LOSS = cal_loss_func_by_OLS(temp_pair_info,global_nomalized_dict)
                    loss_matrix_dict[LOSS] = valence_list
        loss_matrix_dict = self.validate_valence_list_candidates(loss_matrix_dict, res)
        return loss_matrix_dict


    def tune_by_redox_in_certain_range_by_OLS(self, atom_idx_list, LOSS, valence_list, bound, res, global_normalized_normed_dict):
        classes_dict = classify(atom_idx_list,res)
        ori_LOSS = copy.deepcopy(LOSS)
        ori_sum_of_valence = copy.deepcopy(valence_list)

        loss_matrix_dict = self.apply_redox_by_OLS(classes_dict, valence_list, res, global_normalized_normed_dict)

        LOSS = min(loss_matrix_dict.keys())
        valence_list = loss_matrix_dict[LOSS]
        if abs((LOSS - ori_LOSS) / ori_LOSS) > bound:
            return LOSS, valence_list
        else:
            return ori_LOSS, ori_sum_of_valence


###########################################################################################################
###################################################MLE APPROACH############################################
###########################################################################################################
    def apply_redox_by_MLE(self, classes_dict, valence_list, res, global_normalized_normed_dict):
        loss_matrix_dict = {}
        for c in classes_dict: #LOSE electron, valence plus 1
            for d in classes_dict: #get electron, valence minus 1
                if c[0] != d[0]:
                    now_valence_list_i = [valence_list[i] for i in classes_dict[c]]
                    now_max_oxi_list_i = [res.max_oxi_list[i] for i in classes_dict[c]]
                    now_min_oxi_list_i = [max(1,res.min_oxi_list[i]) for i in classes_dict[c]]  #because the center atom must > 0, right!?
                    now_valence_list_j = [valence_list[i] for i in classes_dict[d]]
                    now_max_oxi_list_j = [res.max_oxi_list[i] for i in classes_dict[d]]
                    now_min_oxi_list_j = [max(1,res.min_oxi_list[i]) for i in classes_dict[d]]  #because the center atom must > 0, right!?

                    num_c = len(classes_dict[c])
                    num_d = len(classes_dict[d])
                    lcm = num_c*num_d/math.gcd(num_c,num_d)
                    num_lose = int(lcm/num_c)
                    num_get = int(lcm/num_d)
                    now_valence_list_i = [i + num_lose for i in now_valence_list_i]
                    now_valence_list_j = [i - num_get for i in now_valence_list_j]

                    if all([now_max_oxi_list_i[i] >= now_valence_list_i[i] for i in range(len(classes_dict[c]))]) and all([now_min_oxi_list_j[i] <= now_valence_list_j[i] for i in range(len(classes_dict[d]))]):
                        for i in classes_dict[c]:
                            valence_list[i] += num_lose
                        for i in classes_dict[d]:
                            valence_list[i] -= num_get

                        try:
                            temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                            LOSS = cal_loss_func_by_MLE(temp_pair_info,global_normalized_normed_dict)

                        except:
                            LOSS = 100000
                        loss_matrix_dict[LOSS] = copy.deepcopy(valence_list)
                        for i in classes_dict[c]:
                            valence_list[i] -= num_lose
                        for i in classes_dict[d]:
                            valence_list[i] += num_get
                    else:
                        temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                        LOSS = cal_loss_func_by_MLE(temp_pair_info, global_normalized_normed_dict)
                        loss_matrix_dict[LOSS] = valence_list
                else:
                    temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                    #print(temp_pair_info)
                    LOSS = cal_loss_func_by_MLE(temp_pair_info,global_normalized_normed_dict)
                    loss_matrix_dict[LOSS] = valence_list
        loss_matrix_dict = self.validate_valence_list_candidates(loss_matrix_dict, res)
        #print(loss_matrix_dict)
        return loss_matrix_dict


    def tune_by_redox_in_certain_range_by_MLE(self, atom_idx_list, LOSS, valence_list, bound, res,global_normalized_normed_dict):
        classes_dict = classify(atom_idx_list,res)
        ori_LOSS = copy.deepcopy(LOSS)
        ori_sum_of_valence = copy.deepcopy(valence_list)

        loss_matrix_dict = self.apply_redox_by_MLE(classes_dict, valence_list, res, global_normalized_normed_dict)
        #print(loss_matrix_dict)
        LOSS = max(loss_matrix_dict.keys())
        valence_list = loss_matrix_dict[LOSS]
        if abs((LOSS - ori_LOSS) / ori_LOSS) > bound:
            return LOSS, valence_list
        else:
            return ori_LOSS, ori_sum_of_valence


###########################################################################################################
###################################################MAP APPROACH############################################
###########################################################################################################
    def apply_redox_by_MAP(self, classes_dict, valence_list, res, global_normalized_normed_dict,global_sigma_dict, global_mean_dict):
        loss_matrix_dict = {}
        for c in classes_dict: #LOSE electron, valence plus 1
            for d in classes_dict: #get electron, valence minus 1
                if c[0] != d[0]:
                    now_valence_list_i = [valence_list[i] for i in classes_dict[c]]
                    now_valence_list_j = [valence_list[i] for i in classes_dict[d]]
                    if c[2] == "super":
                        now_max_oxi_list_i = [res.max_oxi_list[i] for i in classes_dict[c]]
                        now_min_oxi_list_i = [max(1,res.min_oxi_list[i]) for i in classes_dict[c]]  #because the center atom must > 0, right!?
                    else: #c[2] == "link"
                        now_min_oxi_list_i = [res.min_oxi_list[i] for i in classes_dict[c]]
                        now_max_oxi_list_i = [min(0, res.max_oxi_list[i]) for i in classes_dict[c]]
                    if d[2] == "super":
                        now_max_oxi_list_j = [res.max_oxi_list[i] for i in classes_dict[d]]
                        now_min_oxi_list_j = [max(1,res.min_oxi_list[i]) for i in classes_dict[d]]  #because the center atom must > 0, right!?
                    else:#d[2] == "link"
                        now_min_oxi_list_j = [res.min_oxi_list[i] for i in classes_dict[d]]
                        now_max_oxi_list_j = [min(0, res.max_oxi_list[i]) for i in classes_dict[d]]                     

                    num_c = len(classes_dict[c])
                    num_d = len(classes_dict[d])
                    lcm = num_c*num_d/math.gcd(num_c,num_d)
                    num_lose = int(lcm/num_c)
                    num_get = int(lcm/num_d)
                    now_valence_list_i = [i + num_lose for i in now_valence_list_i]
                    now_valence_list_j = [i - num_get for i in now_valence_list_j]

                    if all([now_max_oxi_list_i[i] >= now_valence_list_i[i] for i in range(len(classes_dict[c]))]) and all([now_min_oxi_list_j[i] <= now_valence_list_j[i] for i in range(len(classes_dict[d]))]):
                        for i in classes_dict[c]:
                            valence_list[i] += num_lose
                        for i in classes_dict[d]:
                            valence_list[i] -= num_get

                        #try:
                        if True:
                            temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                            LOSS,avg_likelyhood,avg_prior = cal_loss_func_by_MAP(temp_pair_info,global_normalized_normed_dict, global_sigma_dict, global_mean_dict)

                        #except:
                        else:
                            LOSS = 100000
                        loss_matrix_dict[LOSS] = (copy.deepcopy(valence_list), copy.deepcopy(avg_likelyhood), copy.deepcopy(avg_prior))
                        for i in classes_dict[c]:
                            valence_list[i] -= num_lose
                        for i in classes_dict[d]:
                            valence_list[i] += num_get
                    else:
                        temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                        LOSS,avg_likelyhood,avg_prior = cal_loss_func_by_MAP(temp_pair_info, global_normalized_normed_dict, global_sigma_dict, global_mean_dict)
                        loss_matrix_dict[LOSS] = (valence_list,avg_likelyhood,avg_prior)
                else:
                    temp_pair_info = spider_pair_length_with_CN_unnorm(valence_list, res)
                    #print(temp_pair_info)
                    LOSS,avg_likelyhood,avg_prior = cal_loss_func_by_MAP(temp_pair_info,global_normalized_normed_dict, global_sigma_dict, global_mean_dict)
                    loss_matrix_dict[LOSS] = (valence_list,avg_likelyhood,avg_prior)
        loss_matrix_dict = self.validate_valence_list_candidates(loss_matrix_dict, res)
        return loss_matrix_dict


    def tune_by_redox_in_certain_range_by_MAP(self, atom_idx_list, LOSS, likelyhood, prior, valence_list, bound, res, global_normalized_normed_dict, global_sigma_dict, global_mean_dict):
        classes_dict = classify(atom_idx_list,res)
        ori_LOSS = copy.deepcopy(LOSS)
        ori_likelyhood = copy.deepcopy(likelyhood)
        ori_prior = copy.deepcopy(prior)
        ori_sum_of_valence = copy.deepcopy(valence_list)

        loss_matrix_dict = self.apply_redox_by_MAP(classes_dict, 
                                                   valence_list, 
                                                   res, 
                                                   global_normalized_normed_dict, 
                                                   global_sigma_dict, 
                                                   global_mean_dict)
        print(loss_matrix_dict)
        LOSS = min(loss_matrix_dict.keys())
        valence_list = loss_matrix_dict[LOSS][0]
        likelyhood = loss_matrix_dict[LOSS][1]
        prior = loss_matrix_dict[LOSS][2]

        if abs((LOSS - ori_LOSS) / ori_LOSS) > bound:
            return LOSS, likelyhood, prior, valence_list
        else:
            return ori_LOSS, ori_likelyhood, ori_prior, ori_sum_of_valence


    def validate_valence_list_candidates(self, loss_matrix_dict, res):
        refined_dict = {}
        for loss, (valence_list, avg_likelyhood, avg_prior) in loss_matrix_dict.items():
            valid_flag_list = []
            for val, ele in zip(valence_list, res.elements_list):
                if val not in res.dict_ele[ele]["oxi_list"]:
                    valid_flag_list.append(False)
                else:
                    valid_flag_list.append(True)
            if all(valid_flag_list):
                refined_dict[loss] = (valence_list, avg_likelyhood, avg_prior)
        return refined_dict

"""END HERE"""

'END HERE'

In [42]:
import time
import pickle
import os
import random
import openpyxl
import argparse
import copy
import pandas as pd

from get_fos import GET_FOS
from result import RESULT
from post_process import *
from auxilary import *

from multiprocessing.dummy import Pool as ThreadPool
from functools import partial 
import multiprocessing





##############################################################################################################################################################
########################################################## Do the Initial Guess For TOSS #####################################################################
##############################################################################################################################################################


def get_Oxidation_States(m_id,i, atom_pool, server=False, filepath="/", input_tolerance_list=[]):
    if not server:
        tolerance_list = valid_t_dict[m_id]
    else:
        tolerance_list = input_tolerance_list

    GFOS = GET_FOS()
    delta_X = 0.1
    corr_t = []
    ls = time.time()
        
    for t in tolerance_list:
        res = RESULT()
        res.mid = m_id
        TN = TUNE()
        try:
            if not server:
                GFOS.loss_loop(m_id, delta_X, t, tolerance_list, res)
            else:
                res = GFOS.loss_loop(m_id, delta_X, t, tolerance_list, res=None, server=server, filepath=filepath)
            temp_pair_info = spider_pair_length_with_CN_unnorm(res.sum_of_valence, res)

            #now, the matched dict is the global normalization normed dict. 
            loss, likelyhood, prior = cal_loss_func_by_MAP(temp_pair_info, 
                                                           global_normalized_normed_dict, 
                                                           global_sigma_dict, 
                                                           global_mean_dict)
            N_spec = len(res.species_uni_list)
            res.initial_vl = res.sum_of_valence
            
            if len(res.super_atom_idx_list) > 0:
                res.resonance_flag = True
                if res.resonance_flag:
                    avg_LOSS, the_resonance_result = TN.tune_by_resonance(loss,
                                                                          likelyhood,
                                                                          prior,
                                                                          res, 
                                                                          global_normalized_normed_dict,
                                                                          global_sigma_dict, 
                                                                          global_mean_dict)
                    res.final_vl = the_resonance_result[0][0]
                    same_after_resonance = True if res.final_vl == res.initial_vl else False
                    res.sum_of_valence = res.final_vl

                if atom_pool == "super":
                    process_atom_idx_list = res.super_atom_idx_list
                if atom_pool == "link":
                    process_atom_idx_list = res.link_atom_idx_list
                if atom_pool == "all":
                    process_atom_idx_list = res.idx 

                LOSS, LIKELYHOOD, PRIOR, res.final_vl = TN.tune_by_redox_in_certain_range_by_MAP(process_atom_idx_list, 
                                                                                                 loss, 
                                                                                                 likelyhood,
                                                                                                 prior,
                                                                                                 res.sum_of_valence,
                                                                                                 0,
                                                                                                 res,
                                                                                                 global_normalized_normed_dict,
                                                                                                 global_sigma_dict, 
                                                                                                 global_mean_dict)

                res.sum_of_valence = res.final_vl
                same_after_tunation = True if res.final_vl == res.initial_vl else False
                same_after_resonance = True   
            else:
                res.final_vl = res.initial_vl
                same_after_tunation = True
                same_after_resonance = True
                LOSS = loss
                
            parameters = {m_id: [res.resonance_flag, same_after_tunation, same_after_resonance]}
            
            coef = 1.2
            loss_value = coef**N_spec * LOSS
            corr_t.append((t,loss_value,res))
        except Exception as e:
            print(f"An error occurred: {e}")
            None

    try:
        chosen_one = sorted(corr_t, key = lambda x:x[1])[0]
        res = chosen_one[2]
        t = chosen_one[0]

        temp_pair_info_normed = spider_pair_length_with_CN_normed(res)
        temp_pair_info = spider_bond_length(res)

        single_result_dict_normed = {t:temp_pair_info_normed}
        single_result_dict = {t:temp_pair_info}

        OS_result_with_ele = sorted([(i,j) for i,j in zip(res.final_vl, res.elements_list)], key = lambda x :x[1])

        assert all(map(lambda x:res.dict_ele[x[1]]["min_oxi"] <= x[0] <= res.dict_ele[x[1]]["max_oxi"], OS_result_with_ele))

        OS_result = [ij[0] for ij in OS_result_with_ele]

        normalized_single_result_info = normalization(single_result_dict_normed)
        parameters = [m_id, normalized_single_result_info, single_result_dict, OS_result]
        tc = time.time() - ls
        print("Got the Formal Oxidation State of the %sth structure %s in %s seconds."%(i,m_id,tc))
        #print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        #return parameters
    except Exception as e:
        print(f"An error occurred: {e}")
        parameters = [m_id, None, None, None]
        tc = time.time() - ls
        print("Failed to analyze the %sth structure %s in %s seconds."%(i,m_id,tc))

    if not server:
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        return parameters
    else:
        result = pd.DataFrame(np.vstack([np.array(res.elements_list),np.array(res.sum_of_valence),np.array(res.shell_CN_list)]))
        result.index = ["Elements", "Valence","Coordination Number"]
        return result

In [43]:
para = get_Oxidation_States(m_id="1460513.cif",i=0, atom_pool="all")

{9.971905962929593: ([2, -1, -1, -1, -1, 2], 5.6314812339448475, 4.340424728984745), 12.5227277869291: ([1, -1, -1, -1, -1, 3], 5.585407123936165, 6.937320662992934)}
Got the Formal Oxidation State of the 0th structure 1460513.cif in 1.0675404071807861 seconds.
2024-06-04 15:39:09


In [44]:
para

['1460513.cif',
 {('Be', 'B'): {((12, 9), ('2', '-1')): (2.253288088910148, 24.0)},
  ('B', 'B'): {((9, 9), ('-1', '-1')): (1.7065413989690488, 12.0)},
  ('B', 'Ru'): {((9, 12), ('-1', '2')): (2.298401685658856, 24.0)}},
 {1.1: {('Be',
    'B'): {((12, 9), ('2', '-1')): [2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.2549765305447362,
     2.2549765305447362,
     2.2549765305447362,
     2.2553976349663296,
     2.2553976349663296,
     2.2553976349663296,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.2549765305447362,
     2.2549765305447362,
     2.2549765305447362,
     2.2553976349663296,
     2.2553976349663296,
     2.2553976349663296,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763]},
   ('B',
    'B'): {((9, 9), ('-1', '-1')): [1.6651263790340545,
     1.727776207870916,
     1.727776207870916,
     1.7264649081707173,
     

In [ ]:
assert 1==2

# Initial Guess

In [3]:
import numpy as np
import copy
import math
from digest import get_ele_from_sites
from digest import get_propoteries_of_atom
from initialization import get_bond_order
from post_process import enu_permu_combi
from pre_set import CounterSubset
import os
import pandas as pd
import numpy as np
from result import RESULT
from pre_set import PRE_SET
from digest import DIGEST
from get_structure import GET_STRUCTURE
from polyhedron_algo import POLYHEDRON_ALGO
from resonance import RESONANCE
from initialization import *
import sys
sys.path.append("D:/share/TOSS/toss/")

import time
import pickle
import os
import random
import openpyxl
import argparse
import copy
import pandas as pd

#from get_fos import GET_FOS
from result import RESULT
from pre_set import PRE_SET
from digest import DIGEST
from get_structure import GET_STRUCTURE
from post_process import *
from auxilary import *

from multiprocessing.dummy import Pool as ThreadPool
from functools import partial 
import multiprocessing

In [54]:
import numpy as np
import copy
import math
from digest import get_ele_from_sites
from digest import get_propoteries_of_atom
from initialization import get_bond_order
from post_process import enu_permu_combi
from pre_set import CounterSubset


def get_super_atom_list(res):
    super_atom_idx_list = []
    single_atom_idx_list = []
    for i in res.idx:
        ele_c, r_c, X_c, min_oxi_c, max_oxi_c, v_c = get_propoteries_of_atom(i,res.valence_list, res)
        group_idx = [i]
        group_ele = [ele_c]
        group_X = [X_c]
        group_min_oxi = [min_oxi_c]
        group_max_oxi = [max_oxi_c]
        group_v = [v_c]
        for j in res.shell_idx_list[i]:
            ele_n, r_n, X_n, min_oxi_n, max_oxi_n, v_n = get_propoteries_of_atom(j,res.valence_list, res)
            group_idx.append(j)
            group_ele.append(ele_n)
            group_X.append(X_n)
            group_min_oxi.append(min_oxi_n)
            group_max_oxi.append(max_oxi_n)
            group_v.append(v_n)
        #if group_X[0] == min(group_X) and len(group_X) > 2:
        #if group_X[0] == min(group_X) or len(group_X) == 1:
        if group_X[0] < max(group_X):
            super_atom_idx_list.append(i)
        if len(group_X) == 1:
            single_atom_idx_list.append(i)

    organic_super_atom_idx_list = []
    if res.organic_patch:
        for i in reversed(res.idx):
            if get_ele_from_sites(i,res) == "C":
                organic_super_atom_idx_list.append(i)
                if i in super_atom_idx_list:
                    super_atom_idx_list.remove(i)
            if get_ele_from_sites(i,res) == "H" and res.shell_ele_list[i] == ["C"]:
                super_atom_idx_list.remove(i)
                single_atom_idx_list.append(i)#consider the H link to C is the single atom, because its properties is clear.

    return super_atom_idx_list,single_atom_idx_list, organic_super_atom_idx_list


def local_charge_transfer_for_super_atom(i,j,temp_valence_list, delta_X,check_idx_list,res):  
    ele_c, r_c, X_c, min_oxi_c, max_oxi_c, v_c = get_propoteries_of_atom(i,temp_valence_list,res)
    ele_n, r_n, X_n, min_oxi_n, max_oxi_n, v_n = get_propoteries_of_atom(j,temp_valence_list,res)
    v_c = temp_valence_list[i]
    v_n = temp_valence_list[j]

    if max_oxi_c > v_c and v_n > min_oxi_n and res.bo_matrix[i][j] > 0:
        temp_valence_list[i] = v_c + 1
        temp_valence_list[j] = v_n - 1
        res.bo_matrix[i][j] -= 1
        res.bo_matrix[j][i] -= 1
    else:
        if j in check_idx_list:
            check_idx_list.remove(j)
    return temp_valence_list, check_idx_list


def charge_explosive(i,temp_valence_list,delta_X, res):
    check_idx_list = copy.deepcopy(res.shell_idx_list[i])

    flag_ce = True#ce means charge explosive 
    while flag_ce:
        refine_temp_vl = [temp_valence_list[s] for s in check_idx_list]

        temp_X_list = [res.dict_ele[get_ele_from_sites(s,res)]["X"] for s in check_idx_list]

        temp_X_list_with_idx = [[n,m,l] for n,m,l in zip(check_idx_list, temp_X_list, refine_temp_vl)]

        sort_X_list_with_idx = sorted(temp_X_list_with_idx, reverse = True, key = lambda x:[x[2],x[1]])

        ele_c = get_ele_from_sites(i,res)
        X_c = res.dict_ele[ele_c]["X"]

        sort_idx_list = [l[0] for l in sort_X_list_with_idx if l[1] - X_c > delta_X \
                                                            or (ele_c, get_ele_from_sites(l[0],res)) in res.Forced_transfer_group]

        if len(sort_idx_list) > res.max_oxi_list[i]: #and get_ele_from_sites(i,res) in res.periodic_table.transition_metals:
            record = copy.deepcopy(res.max_oxi_list[i])
            res.max_oxi_list[i] = len(sort_idx_list)

        if len(sort_idx_list) > 0:
            j = sort_idx_list[0]

            temp_valence_list, check_idx_list = local_charge_transfer_for_super_atom(i, j,
                                                                                     temp_valence_list,
                                                                                     delta_X,
                                                                                     check_idx_list, 
                                                                                     res)
            if len(check_idx_list) == 0:
                flag_ce = False
        else:
            flag_ce = False
        #print("103", flag_ce)
    try:
        res.max_oxi_list[i] = record 
    except:
        None

    return temp_valence_list


def new_arbitrary_decision(i, temp_valence_list, delta_X, res): 
    ele_i = get_ele_from_sites(i,res)
    for j in res. shell_idx_list[i]:
        ele_j = get_ele_from_sites(j,res)
        if res.dict_ele[ele_i]["X"] - res.dict_ele[ele_j]["X"] > delta_X and abs(temp_valence_list[i]) <= 4 - res.bo_matrix[i][j]:
            temp_valence_list[i] -= res.bo_matrix[i][j]
            temp_valence_list[j] += res.bo_matrix[i][j]

        elif res.dict_ele[ele_j]["X"] - res.dict_ele[ele_i]["X"] > delta_X <= 4 - res.bo_matrix[i][j]:
            temp_valence_list[j] -= res.bo_matrix[i][j]
            temp_valence_list[i] += res.bo_matrix[i][j]  

        else:
            None
    return temp_valence_list


def reassignment_for_link_atom(link_atom_list_temp, sum_of_valence, res): #maybe it is called "reassignment" better. 
    temp_ele_env_v_list = []
    #for i,l,m,n in zip(res.idx, res.elements_list, res.shell_env_list, sum_of_valence):
    for i,l,m,n in zip(res.idx, res.elements_list, res.shell_ele_list, sum_of_valence):
        if i in link_atom_list_temp:
            temp_ele_env_v_list.append([i,l,m,n])
            
    class_of_link = set([(ilmn[1], tuple(sorted(ilmn[2]))) for ilmn in temp_ele_env_v_list])
    class_of_link_with_idx = []

    for cls in class_of_link:
        temp_cls = []
        for ilmn in temp_ele_env_v_list:
            if (ilmn[1], tuple(sorted(ilmn[2]))) == cls:
                temp_cls.append(ilmn)
        class_of_link_with_idx.append(temp_cls)

    for cls in class_of_link_with_idx:
        valence_of_class = [ilmn[3] for ilmn in cls]
        n_e = sum(valence_of_class)
        n_a = len(cls)
        if n_e % n_a == 0:
            quo = n_e/n_a
            for ilmn in cls:
                sum_of_valence[ilmn[0]] = int(quo)
        else:
            quo = n_e // n_a
            rem = n_e % n_a
            for i, ilmn in enumerate(cls):
                if i < rem:
                    sum_of_valence[ilmn[0]] = int(quo+1)
                else:
                    sum_of_valence[ilmn[0]] = int(quo)
    return sum_of_valence


def reassignment_within_shell(super_atom_idx_list, link_atom_idx_list, sum_of_valence, res):
    for i in super_atom_idx_list:
        op_list_temp = res.shell_idx_list[i]
        op_list = []
        for j in op_list_temp:#make sure all link atoms
            if j in link_atom_idx_list:
                op_list.append(j)
        
        under_saturate = []
        for j in op_list:
            if sum_of_valence[j] > res.min_oxi_list[j]:
                for k in res.idx:
                    if sorted(res.shell_ele_list[k]) == sorted(res.shell_ele_list[j]) and k not in under_saturate and \
                                                get_ele_from_sites(k,res) == get_ele_from_sites(j,res):

                        under_n = sum_of_valence[k] - res.min_oxi_list[k]
                        for t in range(under_n):
                            under_saturate.append(k)  #contains the under_saterated atoms and their all same atom.

        over_saturate = []
        for j in op_list:
            if sum_of_valence[j] < res.min_oxi_list[j]:
                for k in res.idx:
                    if sorted(res.shell_ele_list[k]) == sorted(res.shell_ele_list[j]) and k not in over_saturate and \
                                                get_ele_from_sites(k,res) == get_ele_from_sites(j,res):

                        over_n = res.min_oxi_list[k] - sum_of_valence[k]
                        for t in range(over_n):
                            over_saturate.append(k)    #contains the over_saterated atoms and their all same atom.
        
        if len(under_saturate) > 0 and len(over_saturate) > 0:
            n = min(len(under_saturate), len(over_saturate))
        else:
            n = 0
        
        under_saturate_set = list(set(under_saturate))
        over_saturate_set = list(set(over_saturate))

        while n > 0:

            a_temp_X_list = [res.dict_ele[get_ele_from_sites(j,res)]["X"] for j in under_saturate_set]
            a_X_with_id = [(j,x) for j,x in zip(under_saturate_set, a_temp_X_list)]
            a_sorted_X_with_id = sorted(a_X_with_id, key = lambda x: x[1])
   
            acceptor = a_sorted_X_with_id[0][0]

            d_temp_X_list = [res.dict_ele[get_ele_from_sites(j,res)]["X"] for j in over_saturate_set]
            d_X_with_id = [(j,x) for j,x in zip(over_saturate_set, d_temp_X_list)]
            d_sorted_X_with_id = sorted(d_X_with_id, reverse = True, key = lambda x: x[1])

            doner = d_sorted_X_with_id[0][0]

            sum_of_valence[acceptor] -= 1
            sum_of_valence[doner] += 1
            
            if sum_of_valence[acceptor] == res.min_oxi_list[acceptor]:
                under_saturate_set.remove(acceptor)
            if sum_of_valence[doner] == res.min_oxi_list[doner]:
                over_saturate_set.remove(doner)

            n -= 1
            #print("226", n)

    return sum_of_valence


def set_valence_of_link_atoms(link_atom_idx_list, sum_of_valence, arti_n, res):
    check_diff, n = [], 0
    n += arti_n
    
    for l in link_atom_idx_list:
        now_v = sum_of_valence[l]
        now_valid_os = res.dict_ele[get_ele_from_sites(l, res)]["oxi_list"]
        if now_v not in now_valid_os:
            min_valid_os = min([os for os in now_valid_os if os > now_v])
            n += min_valid_os - now_v
            sum_of_valence[l] = min_valid_os

    return n, sum_of_valence


def set_valence_of_super_atoms(super_atom_idx_list, sum_of_valence, n, res):
    for l in super_atom_idx_list:
        now_v = sum_of_valence[l]
        now_valid_os = res.dict_ele[get_ele_from_sites(l, res)]["oxi_list"]
        if now_v not in now_valid_os:
            max_valid_os = max([os for os in now_valid_os if os < now_v])
            n += max_valid_os - now_v
            sum_of_valence[l] = max_valid_os
    return n, sum_of_valence


def validate_solutions(solution_candidate, val, res, over_oxided_switch=False):
    solutions = []
    for solution in solution_candidate:
        flags = []
        for n_charge_back, (class_key, valence_now_for_class) in zip(solution, val.items()):
            if over_oxided_switch:
                valence_later_for_class = valence_now_for_class + n_charge_back
            else:
                valence_later_for_class = valence_now_for_class - n_charge_back
                
            valid_os_for_class = res.dict_ele[class_key[0]]["oxi_list"]
            flags.append(True) if valence_later_for_class in valid_os_for_class else flags.append(False)
        if all([flags]):
            solutions.append(solution)
    return solutions


def enumerate_path(n, super_atom_idx_list, sum_of_valence, res):
    #print("253", sum_of_valence)
    super_atom_ele_list = [get_ele_from_sites(e,res) for e in super_atom_idx_list]
    super_atom_env_list = [tuple(sorted(res.shell_ele_list[e])) for e in super_atom_idx_list]

    super_atom_env_with_ele_list = [(i,j) for i, j in zip(super_atom_ele_list, super_atom_env_list)]
    super_atom_env_with_ele_uni_list = list(set(super_atom_env_with_ele_list))

    num = [0 for i in range(len(super_atom_env_with_ele_uni_list))]
    for i,e in enumerate(super_atom_env_with_ele_uni_list):
        for j in super_atom_env_with_ele_list:
            if e == j:
                num[i] = num[i] + 1

    ex_idx = {}
    for e in super_atom_env_with_ele_uni_list:
        for i,ee in zip(super_atom_idx_list,super_atom_env_with_ele_list):
            if e == ee:
                ex_idx[ee] = i
                break

    val = {}
    for e in super_atom_env_with_ele_uni_list:
        for i,ee in zip(super_atom_idx_list,super_atom_env_with_ele_list):
            if e == ee:
                val[ee] = sum_of_valence[i]
                break

    uni_with_num = [(i,j) for i,j in zip(super_atom_env_with_ele_uni_list, num)]
    num = [i[1] for i in uni_with_num if n//i[1] != 0]             #erase the super atoms whose number less than n!!!!!  
    super_atom_env_with_ele_uni_list = [i[0] for i in uni_with_num if n//i[1] != 0]  #same upper one!!!!
    mathmetical_limit = [n//i for i in num]

    upper_oxi_limit = [val[i] for i in super_atom_env_with_ele_uni_list]

    lower_oxi_limit = [max(res.min_oxi_list[ex_idx[i]], 0) for i in super_atom_env_with_ele_uni_list]

    oxi_limit = [u-l for u,l in zip(upper_oxi_limit, lower_oxi_limit)]

    limit = [min(oxi_limit[i], mathmetical_limit[i]) for i in range(len(super_atom_env_with_ele_uni_list))]

    mtl = 1
    for lmt in limit:
        mtl = mtl * (lmt+1)
    if mtl > 2000000:
        raise ValueError("The number of combination is too large!")

    solution_candidate = enu_permu_combi(limit)

    solution_candidate = [s for s in solution_candidate if np.dot(s, num) == n]
    
    solution = validate_solutions(solution_candidate, val, res)

    PATH_temp = []
    for i in range(len(solution)):
        path = []
        for s,p in zip(solution[i], super_atom_env_with_ele_uni_list):
            ele_here = p[0]
            cur_v = val[p]
            vp = 1
            for t in range(s):
                path.append([res.dict_ele[ele_here]["IP"][cur_v-vp], p[1]])
                vp = vp+1 
        PATH_temp.append(path)      
        
    PATH = []
    for path in PATH_temp:
        sorted_path = sorted(path, reverse = True, key = (lambda x:[x[0],x[1]]))
        PATH.append(sorted_path)

    return PATH


def get_the_operation_path_by_energy(PATH, super_atom_idx_list, sum_of_valence, res):
    if len(PATH) == 1:
        PATH_idx = 0
    else:
        Energy_list = []
        for path_idx, path in enumerate(PATH):
            E = 0
            E_prime = 0
            r_valence_list = copy.deepcopy(sum_of_valence)
            for way in path:
                r_super_atom_ip = [res.dict_ele[get_ele_from_sites(i,res)]["IP"][r_valence_list[i]-1] for i in super_atom_idx_list]
                r_super_atom_env = [tuple(sorted(res.shell_ele_list[i])) for i in super_atom_idx_list]
                r_idx_ip_env = [(i,j,k) for i,j,k in zip(super_atom_idx_list, r_super_atom_ip,r_super_atom_env)]

                for iie in r_idx_ip_env:
                    if iie[1] == way[0] and iie[2] == way[1]:
                        E += way[0]
                        E_prime -= sum([res.dict_ele[ele]["X"] for ele in way[1]])
                        r_valence_list[iie[0]] -= 1

            Energy_list.append((path_idx, round(E,2), round(E_prime,2), -len(path)))
            
        sorted_Energy_list = sorted(Energy_list, key=lambda x:[x[1], x[2],x[3]])

        PATH_idx = sorted_Energy_list[-1][0]

    return PATH_idx


def group_charge_transfer_by_path(PATH,n,super_atom_idx_list,sum_of_valence,res):
    super_atom_IP_list = [res.dict_ele[get_ele_from_sites(e,res)]["IP"][sum_of_valence[e]-1] for e in super_atom_idx_list]

    super_atom_env_list = [tuple(sorted(res.shell_ele_list[e])) for e in super_atom_idx_list]

    super_atom_IP_with_env_and_idx = [[l,m,n] for l,m,n in zip(super_atom_idx_list, 
                                                               super_atom_IP_list,
                                                               super_atom_env_list)]

    way = PATH.pop(0)
    ip = way[0]
    env = way[1]
    operation_list = [l for [l,m,n] in super_atom_IP_with_env_and_idx if m==ip and n==env]
    if len(operation_list) <= n:
        for i in operation_list:
            sum_of_valence[i] = sum_of_valence[i] -1
            n = n - 1
    return n, sum_of_valence


def flatten_link_atom(link_atom_idx_list,sum_of_valence, res):
    temp_env_list = [tuple(sorted(res.shell_ele_list[i])) for i in link_atom_idx_list]
    temp_ele_list = [res.elements_list[i] for i in link_atom_idx_list]
    temp_ele_with_env = [(i,j) for i,j in zip(temp_ele_list,temp_env_list)]
    class_here = set(temp_ele_with_env)
    temp_ele_with_env_with_idx = [(i,j) for i,j in zip(link_atom_idx_list, temp_ele_with_env)]
    arti_n = 0

    for clss in class_here:
        valence_check = []
        op_list = []

        for i in temp_ele_with_env_with_idx:
            if clss == i[1]:
                valence_check.append(sum_of_valence[i[0]])
                op_list.append(i[0])

        if len(set(valence_check)) != 1:
            max_os = max(valence_check)
            arti_n += sum([max_os - valence_check[l] for l in range(len(valence_check))])

            for i in op_list:
                sum_of_valence[i] = max_os

    return arti_n


def over_oxided(link_atom_idx_list, sum_of_valence, n, res):
    N = abs(n)

    species_list = [(res.elements_list[i],tuple(sorted(res.shell_ele_list[i])),sum_of_valence[i]) for i in link_atom_idx_list]
    species_set = list(set(species_list))
    species_dict = {(i[0],i[1]):[] for i in species_set}
    species_now_valence_dict = {(i[0],i[1]):i[2] for i in species_set}

    for j in link_atom_idx_list:
        species_dict[(res.elements_list[j], tuple(sorted(res.shell_ele_list[j])))].append(j)
    
    num_limit = [species_list.count(i) for i in species_set]

    mathmetical_limit = [N//i for i in num_limit]

    fake_link_atom_list = [j for j in link_atom_idx_list if CounterSubset(list(set(link_atom_idx_list)), 
                                                                          list(set(res.shell_idx_list[j])))]
    
    record = {get_ele_from_sites(i,res):res.dict_ele[get_ele_from_sites(i,res)]["X"] for i in fake_link_atom_list}
    max_X = max([res.dict_ele[get_ele_from_sites(i,res)]["X"] for i in link_atom_idx_list])
    for i in fake_link_atom_list:
        res.dict_ele[get_ele_from_sites(i,res)]["X"] = max_X+0.01 

    top_os_limit = [res.max_oxi_list[v[0]] if v[0] in fake_link_atom_list else -1 for k,v in species_dict.items()]
    
    #limit = [i if j[2] == 0 else min(i, -1-j[2]) for i,j in zip(mathmetical_limit, species_set)]
    #limit = [i if j[2] == 0 else min(i, 0-j[2]) for i,j in zip(mathmetical_limit, species_set)]
    #limit = [top if j[2] == 0 else min(i, top-j[2]) for top,i,j in zip(top_os_limit, mathmetical_limit, species_set)]
    limit = [min(i, top-j[2]) for top,i,j in zip(top_os_limit, mathmetical_limit, species_set)]

    
    mtl = 1
    for lmt in limit:
        mtl *= (lmt + 1)
    if mtl > 2000000:
        raise ValueError("The number of combination is too large!")

    solution_candidate = enu_permu_combi(limit)

    solution_candidate = [s for s in solution_candidate if np.dot(s, num_limit) == N]
    
    solutions = validate_solutions(solution_candidate, species_now_valence_dict, res, over_oxided_switch=True)
    
    energy_per_solution = []

    for solution in solutions:
        sum_X = 0
        effective_val = 0
        temp_sum_of_valence = copy.deepcopy(sum_of_valence)
        for i,e in enumerate(solution):
            operation_list = species_dict[(species_set[i][0], species_set[i][1])]
            while e != 0:
                for j in operation_list:
                    effective_val += temp_sum_of_valence[j]
                    temp_sum_of_valence[j] += 1
                    sum_X += res.dict_ele[res.elements_list[j]]["X"]
                e -= 1
                #print("460", e)
        energy_per_solution.append((round(sum_X,2), temp_sum_of_valence, effective_val))

    sum_of_valence = sorted(energy_per_solution, key = lambda x:[x[0],x[2]])[0][1]

    for i in fake_link_atom_list:
        res.dict_ele[get_ele_from_sites(i,res)]["X"] = record[get_ele_from_sites(i,res)] 

    return sum_of_valence


def operate_bo_matrix(super_atom_idx_list,organic_super_atom_idx_list, res):#only apply this in the organic structures.
    for i in super_atom_idx_list:
        if len(res.shell_ele_list[i]) > 1:
            for j in res.shell_idx_list[i]:
                if j in super_atom_idx_list:
                    res.bo_matrix[i][j] = 0
                #if j in organic_super_atom_idx_list:
                    #res.bo_matrix[i][j] = 0
                    #res.bo_matrix[j][i] = 0
        else:
            None #like H only link to one atom, it must lose one electron to that atom.
    
    octet_dict = {"S":{"top":6, "bottom":2}, 
                  "C":{"top":4, "bottom":4}, 
                  "N":{"top":5, "bottom":3}, 
                  "P":{"top":5, "bottom":3}} #super_atom_idx_list, single_atom_idx, organic_super_atom_idx_list

    organic_elements = list(octet_dict.keys()) + ["O","H"]
    sum_bo_list = []
    for i in res.idx:
        temp_sum_bo = sum(res.bo_matrix[i])
        for j in res.shell_idx_list[i]:
            if get_ele_from_sites(j,res) not in organic_elements: #octet rules + HO
                temp_sum_bo -= 1
        sum_bo_list.append(temp_sum_bo)

    for i in res.idx:
        abs_sum_of_bo = 0
        for j in res.shell_idx_list[i]:
            bo,_ = get_bond_order(i,j,res)

            if res.dict_ele[get_ele_from_sites(j,res)]["X"] >= res.dict_ele[get_ele_from_sites(i,res)]["X"]:
                abs_sum_of_bo += bo
            else:
                abs_sum_of_bo -= bo
        SIGN = "top" if abs_sum_of_bo >= 0 else "bottom"
            
        ele_i = get_ele_from_sites(i,res)
        if ele_i in list(octet_dict.keys()) and sum_bo_list[i] > octet_dict[ele_i][SIGN]:
            organic_shell_idx_list = [j for j in res.shell_idx_list[i] if get_ele_from_sites(j,res) in organic_elements] #octet rules + HO
            sum_bo_check = [(get_ele_from_sites(j,res), res.bo_matrix[i][j]) for j in organic_shell_idx_list]
            
            if sum_bo_check.count(("O",2)) > 1:
                idx_with_env_ele_bo = [(j, 
                                        res.shell_env_list[j], 
                                        get_ele_from_sites(j,res), 
                                        res.bo_matrix[i][j]) for j in organic_shell_idx_list]

                O_info = [info for info in idx_with_env_ele_bo if info[2] == "O" and info[3] == 2]
                sort_O_info = sorted(O_info, reverse = True, key = lambda x:x[1]) 
                j = sort_O_info[0][0]
                res.bo_matrix[i][j] -= 1
                res.bo_matrix[j][i] -= 1
                sum_bo_list[i] -= 1
                sum_bo_list[j] -= 1

            elif sum_bo_check.count(("O",2)) == 1 and sum_bo_check.count(("C",2)) == 1:
                idx_with_env_ele_bo = [(j, 
                                        res.shell_env_list[j], 
                                        get_ele_from_sites(j,res), 
                                        res.bo_matrix[i][j]) for j in organic_shell_idx_list]

                C_idx = [info for info in idx_with_env_ele_bo if info[2] == "C" and info[3] == 2][0][0]
                O_idx = [info for info in idx_with_env_ele_bo if info[2] == "O" and info[3] == 2][0][0]
                if sum_bo_list[C_idx] > octet_dict[ele_i]["top"]:
                    j = C_idx
                else:
                    j = O_idx
                res.bo_matrix[i][j] -= 1
                res.bo_matrix[j][i] -= 1
                sum_bo_list[i] -= 1
                sum_bo_list[j] -= 1

            else:
                if get_ele_from_sites(i,res) != "C":
                    for j in organic_shell_idx_list:
                            ele_j = get_ele_from_sites(j,res)
                            if ele_j in list(octet_dict.keys()) and sum_bo_list[j] > octet_dict[ele_j][SIGN]:
                                if res.bo_matrix[i][j] == 2:
                                    sum_bo_list[i] -= 1
                                    sum_bo_list[j] -= 1
                                    res.bo_matrix[i][j] -= 1
                                    res.bo_matrix[j][i] -= 1
                                    break

        if ele_i in list(octet_dict.keys()) and sum_bo_list[i] < octet_dict[ele_i][SIGN]:
            organic_shell_idx_list = [j for j in res.shell_idx_list[i] if get_ele_from_sites(j,res) in organic_elements]
            for j in organic_shell_idx_list:
                if get_ele_from_sites(j,res) == "C":
                    if res.bo_matrix[i][j] == 1:
                        sum_bo_list[i] += 1
                        sum_bo_list[j] += 1
                        res.bo_matrix[i][j] += 1
                        res.bo_matrix[j][i] += 1
                        break                  
    return None


def retrive_oxi(res, super_atom_idx_list, link_atom_idx_list):
    for j in link_atom_idx_list:
        if res.min_oxi_list[j] == 0:
            #the ability of recepting electrons should be retrived!
            #retrived = len(set(res.covalent_pair[j]))
            retrived = res.original_min_oxi_list[j]
            res.min_oxi_list[j] = retrived

    for i in super_atom_idx_list:
        if res.max_oxi_list[i] == 0:
            res.max_oxi_list[i] = res.dict_ele[get_ele_from_sites(i,res)]["max_oxi"]
    return None


def break_bond(res,i,j):
    shell_idx_list[i].remove(j)
    shell_CN_list[i] = len(shell_idx_list[i])
    shell_ele_list[i] = [get_ele_from_sites(k,res) for k in shell_idx_list[i]]
    shell_X_list[i] = [res.dict_ele[get_ele_from_sites(k,res)]["X"] for k in shell_idx_list[i]]
    shell_env_list[i] = round(sum(shell_idx_list[i]),2)


def link_bond(res,ori_res,i):
    res.shell_idx_list[i] = copy.deepcopy(ori_res.shell_idx_list[i])
    res.shell_CN_list[i] = copy.deepcopy(ori_res.shell_CN_list[i])
    res.shell_ele_list[i] = copy.deepcopy(ori_res.shell_ele_list[i])
    res.shell_X_list[i] = copy.deepcopy(ori_res.shell_X_list[i])
    res.shell_env_list[i] = copy.deepcopy(ori_res.shell_env_list[i])


def cation_bond(res,sum_of_valence):
    waiting_for_break = []
    for i in res.idx:
        for j in res.shell_idx_list[i]:
            if sum_of_valence[i] > 0 and sum_of_valence[j] > 0:
                if get_ele_from_sites(i,res) in res.periodic_table.metals and get_ele_from_sites(j,res) in res.periodic_table.metals:
                    waiting_for_break.append((i,j))
                else:
                    None
            else:
                None

    #the fake ZERO valence!
    for i in res.idx:
        for j in res.shell_idx_list[i]:
            a = sum_of_valence[i]
            b = sum_of_valence[j]
            Xa = res.dict_ele[get_ele_from_sites(i,res)]["X"]
            Xb = res.dict_ele[get_ele_from_sites(j,res)]["X"]
            if a + b > 0 and a * b == 0:
                if abs(Xa - Xb) > 0.1:
                    if get_ele_from_sites(i,res) in res.periodic_table.metals and get_ele_from_sites(j,res) in res.periodic_table.metals:
                        waiting_for_break.append((i,j))
                    else:
                        None
                else:
                    None
            else:
                    None

    Cation_bond = True if waiting_for_break != [] else False

    #for pair in waiting_for_break:
        #break_bond(res, pair[0], pair[1])

    #if [] in res.shell_idx_list:
        #raise ValueError("Superatom is isolated!")

    #res.bo_matrix = copy.deepcopy(res.ori_bo_matrix)

    return Cation_bond, waiting_for_break


def process_cation_bond(break_pair_list, res):
    if break_pair_list != []:

        for pair in break_pair_list:
            i = pair[0]
            j = pair[1]
            
            res.shell_idx_list[i].remove(j)

        break_flag = False
        repair = []
        for i, idx_list in enumerate(res.shell_idx_list):
            if idx_list == []:
                repair += [pair for pair in break_pair_list if pair[0] == i]
                break_flag = True
            else:
                None
        for pair in repair:
            i = pair[0]
            j = pair[1]
            res.shell_idx_list[i].append(j)
            res.shell_idx_list[j].append(i)

        for i in res.idx:
            res.shell_ele_list[i] = [get_ele_from_sites(j,res) for j in res.shell_idx_list[i]]
            res.shell_CN_list[i] = len(res.shell_ele_list[i])
            res.shell_X_list[i] = [res.dict_ele[ele]["X"] for ele in res.shell_ele_list[j]]
            res.shell_env_list[i] = round(sum(res.shell_X_list[i]),2)
    else:
        break_flag = False

    res.bo_matrix = copy.deepcopy(res.ori_bo_matrix)
    return break_flag


def lower_oxidation_level(res, sum_of_valence, link_atom_idx_list, n, lower_recording):
    sl_pair_list = []
    operating_link_atom_list = copy.deepcopy(link_atom_idx_list)

    for j in reversed(operating_link_atom_list): # Backword enumerate link atom list, why?
        if sum_of_valence[j] == -1:
            operating_link_atom_list.remove(j) # DO NOT consider the -1 valence link atom, which answer the question backward enumerating.
    
    for j in operating_link_atom_list: # Shorter list
        for i in res.shell_idx_list[j]:
            ele_j = get_ele_from_sites(j,res)
            ele_i = get_ele_from_sites(i,res)
            sl_pair_list.append((j, ele_i, ele_j, round(abs(res.dict_ele[ele_j]["X"] - res.dict_ele[ele_i]["X"]),2), lower_recording[j]["level"]))
            # sl_pait_list[idx] = (index_j, ele_i, ele_j, diff_X, lower_level)

    sort_sl_pair_list = sorted(list(sl_pair_list), key = lambda x:(x[4],x[3])) # sort the list by diff_X and lover_level

    min_delta_x = sort_sl_pair_list[0][3] # Minimum diff_X

    op_link_atom_set = set([p[0] for p in sort_sl_pair_list if p[3] == min_delta_x]) # only operate the minimum diff_X pair

    for j in op_link_atom_set:
        lower_recording[j]["level"] += 1
        res.min_oxi_list[j] += 1
        lower_recording[j]["min"] += 1
        sum_of_valence[j] += 1
        n += 1

    return n, sum_of_valence, lower_recording


def jump_to_resonance(res, sum_of_valence, super_atom_idx_list, n):
    print("cool")
    alkali = res.periodic_table.alkali
    earth_alkali = res.periodic_table.earth_alkali

    # Combine alkali and earth alkali elements into one dictionary
    set_value_ele_dict = {**alkali, **earth_alkali}
    
    # Find resonance atoms in super_atom_idx_list
    resonance_atoms = [i for i in super_atom_idx_list if get_ele_from_sites(i, res) in set_value_ele_dict]
    
    if resonance_atoms:
        while n > 0:
            # Sort super_atom_idx_list by sum_of_valence in descending order
            sorted_idx_val = sorted(super_atom_idx_list, key=lambda i: sum_of_valence[i], reverse=True)
            op_i = sorted_idx_val[0]  # Get the index with the highest valence sum
            
            ele_i = get_ele_from_sites(op_i, res)
            valid_os_ele_i = res.dict_ele[ele_i]["oxi_list"]
            
            # Find the maximum valid oxidation state less than current valence sum
            v = max(os for os in valid_os_ele_i if os < sum_of_valence[op_i])
            
            # Decrease n by the difference and update sum_of_valence
            n -= (sum_of_valence[op_i] - v)
            sum_of_valence[op_i] = v
        
        return True
    return False

class POLYHEDRON_ALGO():
    def __init__(self, delta_X, res):
        #res.alloy_flag = False
        if res.alloy_flag:
            n, sum_of_valence, super_atom_idx_list, link_atom_idx_list, single_atom_idx_list = 0,[0 for o in res.idx],[],[],[]
            self.ori_n,self.ori_super_atom_idx_list,self.ori_sum_of_valence = 0, [], []
        else:
            Cation_Bond = True
            waiting_for_break = []

            while Cation_Bond:
                break_flag = process_cation_bond(waiting_for_break, res)

                if break_flag:
                    break
                else:
                    None

                sum_of_valence,\
                super_atom_idx_list,\
                single_atom_idx_list,\
                organic_super_atom_idx_list,\
                link_atom_idx_list = self.main_part(res, delta_X)

                Cation_Bond, waiting_for_break = cation_bond(res, sum_of_valence)

        self.super_atom_idx_list = super_atom_idx_list
        self.single_atom_idx_list = single_atom_idx_list
        self.link_atom_idx_list = link_atom_idx_list
        self.sum_of_valence = sum_of_valence
        

    def main_part(self, res, delta_X):
        sum_of_valence = [0 for l in res.idx]
        super_atom_idx_list,\
        single_atom_idx_list,\
        organic_super_atom_idx_list = get_super_atom_list(res)

        link_atom_idx_list = list(set(res.idx).difference(set(super_atom_idx_list)))
        retrive_oxi(res, super_atom_idx_list, link_atom_idx_list)

        if res.organic_patch:
            operate_bo_matrix(super_atom_idx_list,organic_super_atom_idx_list, res)

        for i in super_atom_idx_list:
            temp_valence_list = [0 for l in res.idx]
            temp_valence_list = charge_explosive(i, temp_valence_list, delta_X, res)
            print(temp_valence_list)
            sum_of_valence = np.sum([sum_of_valence, temp_valence_list], axis = 0).tolist()

        if res.organic_patch:
            for i in organic_super_atom_idx_list:
                temp_valence_list = [0 for l in res.idx]
                temp_valence_list = new_arbitrary_decision(i, temp_valence_list, delta_X, res)
                sum_of_valence = np.sum([sum_of_valence, temp_valence_list], axis = 0).tolist()
            # new adding
            for i in reversed(organic_super_atom_idx_list):
                if sum_of_valence[i] < res.min_oxi_list[i]:
                    organic_super_atom_idx_list.remove(i)
                if sum_of_valence[i] > res.max_oxi_list[i]:
                    super_atom_idx_list.append(i)
                    
        for i in reversed(super_atom_idx_list):
            if sum_of_valence[i] < 0:
                super_atom_idx_list.remove(i)
        print(sum_of_valence)

        link_atom_idx_list = list(set(res.idx).difference(set(super_atom_idx_list)))
        link_atom_idx_list = list(set(link_atom_idx_list).difference(set(single_atom_idx_list)))
        link_atom_idx_list = list(set(link_atom_idx_list).difference(set(organic_super_atom_idx_list)))

        sum_of_valence = reassignment_within_shell(super_atom_idx_list,
                                                   link_atom_idx_list,
                                                   sum_of_valence, 
                                                   res)

        sum_of_valence = reassignment_within_shell(organic_super_atom_idx_list,
                                                   link_atom_idx_list,
                                                   sum_of_valence, 
                                                   res)

        sum_of_valence = reassignment_for_link_atom(link_atom_idx_list,
                                                    sum_of_valence, 
                                                    res)

        arti_n  = flatten_link_atom(link_atom_idx_list,sum_of_valence, res)

        n, sum_of_valence = set_valence_of_link_atoms(link_atom_idx_list, sum_of_valence, arti_n, res) 

        n, sum_of_valence = set_valence_of_super_atoms(super_atom_idx_list, sum_of_valence, n, res) 

        for i in res.idx:
            if get_ele_from_sites(i,res) == "H" and sum_of_valence[i] == 1:
                if i in reversed(super_atom_idx_list):
                    super_atom_idx_list.remove(i)

        #super_atom_idx_list = list(set(super_atom_idx_list).difference(set(res.inorganic_acid_center_idx)))

        self.ori_n = copy.deepcopy(n)
        self.ori_super_atom_idx_list = copy.deepcopy(super_atom_idx_list)
        self.ori_sum_of_valence = copy.deepcopy(sum_of_valence)
        
        n, sum_of_valence = 6, [2, -1, -1, -1, -1, 8]
        
        if n > 0:
            if len(super_atom_idx_list) == 0:
                raise KeyError("There is NO Super Atom anymore!")

            PATH = enumerate_path(n,super_atom_idx_list,sum_of_valence,res)
            PATH = []

            if len(PATH) == 0:
                jump_flag = jump_to_resonance(res,sum_of_valence,super_atom_idx_list, n)
                if jump_flag:
                    return sum_of_valence, super_atom_idx_list, single_atom_idx_list, organic_super_atom_idx_list, link_atom_idx_list
                else:
                    None

            lower_recording = {j:{"level":0, "min":res.min_oxi_list[j]} for j in link_atom_idx_list}

            #while len(PATH) == 0 and [sum_of_valence[j] for j in link_atom_idx_list].count(-1) != len(link_atom_idx_list):
            all_lg_minus1 = list(filter(lambda x:x>=-1, [sum_of_valence[j] for j in link_atom_idx_list]))

            while len(PATH) == 0 and len(all_lg_minus1) < len(link_atom_idx_list):
                # FIND a path or ALL link atoms' valences change to -1, stop the while!

                n, sum_of_valence, lower_recording = lower_oxidation_level(res, 
                                                                          sum_of_valence, 
                                                                          link_atom_idx_list,
                                                                          n, 
                                                                          lower_recording)

                PATH = enumerate_path(n,super_atom_idx_list,sum_of_valence,res)

                all_lg_minus1 = list(filter(lambda x:x>=-1, [sum_of_valence[j] for j in link_atom_idx_list]))

            if len(PATH) ==0:
                raise ValueError("There is no porper way for putting charge back!")
            else:
                None #FIND the proper way!
                

            lowest_energy_PATH_idx = get_the_operation_path_by_energy(PATH, super_atom_idx_list, sum_of_valence, res)

            while n > 0:
                n, sum_of_valence = group_charge_transfer_by_path(PATH[lowest_energy_PATH_idx],
                                                                  n,
                                                                  super_atom_idx_list,
                                                                  sum_of_valence,
                                                                  res)
                #print("872",n)
                
        if n < 0:
            sum_of_valence = over_oxided(link_atom_idx_list, sum_of_valence, n, res)

        return sum_of_valence, super_atom_idx_list, single_atom_idx_list, organic_super_atom_idx_list, link_atom_idx_list

    def uniformity(self,vl,res, atom_list):
        ele_list = [get_ele_from_sites(i,res) for i in atom_list]
        env_list = [tuple(sorted(res.shell_ele_list[i])) for i in atom_list]
        val_list = [vl[i] for i in atom_list]
        species_list = [(i,j,k) for i,j,k in zip(ele_list, env_list, val_list)]
        species_uni_list = list(set(species_list))
        return species_uni_list


In [59]:
class GET_FOS():

    def initial_guess(self, m_id, delta_X, tolerance, tolerance_list, res, server=False, filepath="/"):
        if server:
            res = RESULT()
            filepath, m_id = os.path.split(filepath)

        PS = PRE_SET()
        res.dict_ele,\
        res.matrix_of_threshold = PS.dict_ele, PS.matrix_of_threshold 

        res.Forced_transfer_group,\
        res.inorganic_group = PS.Forced_transfer_group, PS.inorganic_group
        #res.transit_metals, res.metals = PS.transit_matals, PS.metals

        if not server:
            GS = GET_STRUCTURE(m_id, "D:/share/TOSS_2024/structures/")
        else:
            GS = GET_STRUCTURE(m_id, specific_path="../toss_server/%s/"%filepath)
        res.sites,\
        res.idx, \
        res.struct = GS.sites, GS.idx, GS.struct

        res.matrix_of_length,\
        res.valence_list, \
        res.elements_list = GS.matrix_of_length, GS.valence_list, GS.elements_list

        benchmark_tolerance, \
        tolerance_list = self.nearest(tolerance, tolerance_list)

        DG = DIGEST(tolerance_list, tolerance, m_id, res)
        res.max_oxi_list, \
        res.min_oxi_list = DG.max_oxi_list, DG.min_oxi_list

        res.SHELL_idx_list, \
        res.threshold_list,\
        res.SHELL_idx_list_with_images = DG.SHELL_idx_list, DG.threshold_list, DG.SHELL_idx_list_with_images

        res.organic_patch, \
        res.alloy_flag = DG.organic_patch, DG.alloy_flag

        DG.digest_structure_with_image(res)
        res.shell_ele_list, \
        res.shell_env_list, \
        res.shell_idx_list, \
        res.shell_CN_list, \
        res.shell_X_list = DG.shell_ele_list, DG.shell_env_list, DG.shell_idx_list, DG.shell_CN_list, DG.shell_X_list
        
        INT = INITIAL(tolerance, m_id, delta_X, res)
        res.bo_matrix, \
        res.print_covalent_status, \
        res.ori_bo_matrix, \
        res.covalent_pair, \
        res.original_min_oxi_list, \
        res.original_max_oxi_list = INT.bo_matrix, INT.print_covalent_status, INT.ori_bo_matrix, INT.covalent_pair, INT.original_min_oxi_list, INT.original_max_oxi_list 
        
        LA = LOCAL_ALGO(m_id, delta_X, tolerance, res)
        res.inorganic_acid_flag, \
        res.first_algorithm_flag = LA.inorganic_acid_flag, LA.first_algorithm_flag

        LA.apply_local_iter_method(res)
        res.inorganic_acid_center_idx = LA.inorganic_acid_center_idx

        LA.apply_local_algo(delta_X,res)
        
        PA = POLYHEDRON_ALGO(delta_X, res)
        res.ori_n,\
        res.ori_super_atom_idx_list,\
        res.ori_sum_of_valence = PA.ori_n,PA.ori_super_atom_idx_list,PA.ori_sum_of_valence

        res.sum_of_valence,\
        res.super_atom_idx_list,\
        res.link_atom_idx_list,\
        res.single_atom_idx_list = PA.sum_of_valence,PA.super_atom_idx_list,PA.link_atom_idx_list,PA.single_atom_idx_list

        #res.sum_of_valence = [0 ,-1, -1, -1, 4]
        
        RSN = RESONANCE(res)
        res.resonance_flag, \
        res.resonance_order = RSN.resonance_flag, RSN.resonance_order   

        if res.resonance_flag:
            print("cool 81")
            res.fake_n, \
            res.exclude_super_atom_list, \
            res.perfect_valence_list, \
            res.plus_n = RSN.fake_n, RSN.exclude_super_atom_list, RSN.perfect_valence_list, RSN.plus_n,

            RSN.erase_bg_charge(res)
        if not server:
            res.species_uni_list = PA.uniformity(res.sum_of_valence, res, res.idx)
        else:
            res.species_uni_list = PA.uniformity(res.sum_of_valence, res, res.idx)
            result = pd.DataFrame(np.vstack([np.array(res.elements_list),np.array(res.sum_of_valence),np.array(res.shell_CN_list)]))
            result.index = ["Elements", "Valence","Coordination Number"]
            return result


    def loss_loop(self, m_id, delta_X, tolerance, tolerance_list, res, server=False, filepath="/"):
        if server:
            res = RESULT()
            filepath, m_id = os.path.split(filepath)

        PS = PRE_SET()
        res.dict_ele,\
        res.matrix_of_threshold = PS.dict_ele, PS.matrix_of_threshold 

        res.Forced_transfer_group,\
        res.inorganic_group = PS.Forced_transfer_group, PS.inorganic_group#PS.inorganic_group  #get rid of the effect of the inorganic group.
        #res.transit_metals, res.metals = PS.transit_matals, PS.metals

        if not server:
            GS = GET_STRUCTURE(m_id)
        else:
            GS = GET_STRUCTURE(m_id, specific_path="../toss_server/%s/"%filepath)
        res.sites,\
        res.idx, \
        res.struct = GS.sites, GS.idx, GS.struct

        res.matrix_of_length, \
        res.valence_list, \
        res.elements_list = GS.matrix_of_length, GS.valence_list, GS.elements_list

        benchmark_tolerance, \
        tolerance_list = self.nearest(tolerance, tolerance_list)

        DG = DIGEST(tolerance_list, tolerance, m_id, res)
        res.max_oxi_list, \
        res.min_oxi_list = DG.max_oxi_list, DG.min_oxi_list

        res.SHELL_idx_list, \
        res.threshold_list = DG.SHELL_idx_list, DG.threshold_list

        res.SHELL_idx_list_with_images = DG.SHELL_idx_list_with_images

        res.organic_patch, \
        res.alloy_flag = DG.organic_patch, DG.alloy_flag

        DG.digest_structure_with_image(res)
        res.shell_ele_list, \
        res.shell_env_list, \
        res.shell_idx_list, \
        res.shell_CN_list, \
        res.shell_X_list = DG.shell_ele_list, DG.shell_env_list, DG.shell_idx_list, DG.shell_CN_list, DG.shell_X_list
        
        INT = INITIAL(tolerance, m_id, delta_X, res)
        res.bo_matrix, \
        res.print_covalent_status, \
        res.ori_bo_matrix, \
        res.covalent_pair, \
        res.original_min_oxi_list, \
        res.original_max_oxi_list = INT.bo_matrix, INT.print_covalent_status, INT.ori_bo_matrix, INT.covalent_pair, INT.original_min_oxi_list, INT.original_max_oxi_list 

        LA = LOCAL_ALGO(m_id, delta_X, tolerance, res)
        res.inorganic_acid_flag,\
        res.first_algorithm_flag = LA.inorganic_acid_flag, LA.first_algorithm_flag

        LA.apply_local_iter_method(res)
        res.inorganic_acid_center_idx = LA.inorganic_acid_center_idx
        LA.apply_local_algo(delta_X,res)

        PA = POLYHEDRON_ALGO(delta_X, res)
        res.ori_n,\
        res.ori_super_atom_idx_list,\
        res.ori_sum_of_valence = PA.ori_n,PA.ori_super_atom_idx_list,PA.ori_sum_of_valence

        res.sum_of_valence,\
        res.super_atom_idx_list,\
        res.link_atom_idx_list,\
        res.single_atom_idx_list = PA.sum_of_valence,PA.super_atom_idx_list,PA.link_atom_idx_list,PA.single_atom_idx_list
        
        RSN = RESONANCE(res)
        res.resonance_flag, \
        res.resonance_order = RSN.resonance_flag, RSN.resonance_order       

        res.species_uni_list = PA.uniformity(res.sum_of_valence, res, res.super_atom_idx_list)
        if server:
            return res
        

    def nearest(self, tolerance, tolerance_list):

        if tolerance_list[-1] <= tolerance:
            benchmark_t = tolerance_list[-1]
        elif tolerance_list[0] >= tolerance:
            benchmark_t = tolerance_list[0]
        else:
            for i,t in enumerate(tolerance_list):
                if t > tolerance:
                    break
            benchmark_t = tolerance_list[i-1]
        refined_tolerance_list = [t for t in tolerance_list if t <= tolerance]
        return benchmark_t, refined_tolerance_list


In [60]:
def tolerance_corr(func, m_id, delta_X, tolerance_list):
    tolerance_trial = tolerance_list
    single_result_dict_normed = {}
    single_result_dict = {}
    single_super_point_dict = {}
    for t in tolerance_trial:
        try:
            res = RESULT()
            res.mid = m_id
            func(m_id, delta_X, t, tolerance_list, res)
            temp_pair_info_normed = spider_pair_length_with_CN_normed(res)
            temp_pair_info = spider_bond_length(res)
            single_result_dict[t] = temp_pair_info
            single_result_dict_normed[t] = temp_pair_info_normed
            super_point_list = [[get_ele_from_sites(i,res), sorted(res.shell_ele_list[i])] for i in res.idx]
            single_super_point_dict[t] = super_point_list
        except Exception as e:  
            print(f"An error occurred: {e}")
            LOSS = None
            temp_pair_info = None
            temp_pair_info_normed = None
            super_point_list = None
    return single_result_dict_normed, single_result_dict, single_super_point_dict #They could be empty.


def get_Initial_Guess(m_id, i):
    GFOS = GET_FOS()
    delta_X = 0.1

    tolerance_list = valid_t_dict[m_id]

    ls = time.time()
    try:
        single_result_dict_normed, single_result_dict, single_super_point_dict = tolerance_corr(GFOS.initial_guess, 
                                                                                                m_id, 
                                                                                                delta_X, 
                                                                                                tolerance_list)

        if single_result_dict_normed != {}:
            normalized_single_result_info = normalization(single_result_dict_normed)
        else:
            raise ValueError
        parameters = [m_id, tolerance_list, normalized_single_result_info, single_result_dict]
        tc = time.time() - ls
        print("Got the Formal Oxidation State of the %sth structure %s in %s seconds."%(i,m_id,tc))
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        return parameters
    except Exception as e:   
        print(f"An error occurred: {e}")
        parameters = [m_id, tolerance_list, None, None]
        tc = time.time() - ls
        print("Failed to analyze the %sth structure %s in %s seconds."%(i,m_id,tc))
        print(time.strftime('%Y-%m-%d %H:%M:%S', time.localtime()))
        return parameters


In [61]:
para = get_Initial_Guess("1460513.cif",0)

[8, -2, -2, -2, -2, 0]
[0, -2, -2, -2, -2, 8]
[8, -4, -4, -4, -4, 8]
cool
cool 66
[2, -1, -1, -1, -1, 2]
Got the Formal Oxidation State of the 0th structure 1460513.cif in 0.20414113998413086 seconds.
2024-06-04 13:34:12


In [58]:
para

['1460513.cif',
 [1.1],
 {('Be', 'B'): {((12, 9), ('2', '-1')): (2.253288088910148, 24.0)},
  ('B', 'B'): {((9, 9), ('-1', '-1')): (1.7065413989690488, 12.0)},
  ('B', 'Ru'): {((9, 12), ('-1', '2')): (2.298401685658856, 24.0)}},
 {1.1: {('Be',
    'B'): {((12, 9), ('2', '-1')): [2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.2549765305447362,
     2.2549765305447362,
     2.2549765305447362,
     2.2553976349663296,
     2.2553976349663296,
     2.2553976349663296,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763,
     2.2549765305447362,
     2.2549765305447362,
     2.2549765305447362,
     2.2553976349663296,
     2.2553976349663296,
     2.2553976349663296,
     2.251389095064763,
     2.251389095064763,
     2.251389095064763]},
   ('B',
    'B'): {((9, 9), ('-1', '-1')): [1.6651263790340545,
     1.727776207870916,
     1.727776207870916,
     1.726464908170717

In [9]:
assert 1==2

AssertionError: 

In [ ]:
from pre_set import PRE_SET
from result import RESULT

PS = PRE_SET()
res = RESULT()

elements_values = sorted([(e,PS.dict_ele[e]["X"]) for e in PS.dict_ele.keys()], key=lambda x:x[1])
metals = res.periodic_table.metals

MM_values = sorted([(e,PS.dict_ele[e]["X"]) for e in PS.dict_ele.keys() if e in metals], key=lambda x:x[1])
NM_values = sorted([(e,PS.dict_ele[e]["X"]) for e in PS.dict_ele.keys() if e not in metals], key=lambda x:x[1])

In [ ]:
import matplotlib.pyplot as plt

elements, values = zip(*elements_values)# Separate the elements and their values
MMelements, MMvalues = zip(*MM_values)
NMelements, NMvalues = zip(*NM_values)


# Create the horizontal bar chart
fig = plt.figure(figsize=(10, 20))
bars = plt.barh(elements, values, color='skyblue')
bars1 = plt.barh(MMelements, MMvalues, color='skyblue')
bars2 = plt.barh(NMelements, NMvalues, color='red')

# Add value labels on the bars
for bar, value in zip(bars, values):
    xval = bar.get_width()
    plt.text(xval, bar.get_y() + bar.get_height()/2, round(value, 2), va='center')

plt.ylabel('Elements')
plt.xlabel('Values')
plt.title('Horizontal Bar Chart of Elements and Their Values')
plt.show()
fig.savefig("../toss_nb/x.PNG")

In [52]:
import pandas as pd
import numpy as np
import pickle
from scipy.optimize import curve_fit
from sklearn.metrics import r2_score, mean_squared_error
from matplotlib.gridspec import GridSpec

OV = pd.read_csv("../redefined_length_csv/bond_of_O_V.csv", header=None)[0].tolist()
OAl = pd.read_csv("../redefined_length_csv/bond_of_O_Al.csv", header=None)[0].tolist()

with open("../fitting_info_dict.pkl", "rb") as f:
    fitting_info_dict = pickle.load(f)


def refine_data(Min, Max, step, target_list):
    stations = np.arange(Min, Max + step, step)
    stations = np.round(stations, 3)
    def find_station(value):
        station_indices = np.searchsorted(stations, value, side='right') - 1
        return stations[max(station_indices, 0)]
    processed_list = [find_station(value) for value in target_list]
    return processed_list

def length_2_distribution(length, t):
    length = [l for l in length if t >= l > 0.1]
    Min, Max = min(length), max(length)
    Step = round(((Max-Min)/50),3)
    refined_distance = refine_data(Min, Max, Step, length)
    unique_values, counts = np.unique(refined_distance, return_counts=True)
    return unique_values, counts/sum(counts)


def gaussian(x, amplified1, mean1, stddev1, amplified2, mean2, stddev2):
    return amplified1 * np.exp((-(x-mean1)**2)/(2*(stddev1**2))) + amplified2 * np.exp((-(x-mean2)**2)/(2*(stddev2**2)))


def test_r2(x, y, mean1, mean2, step, MIN,MAX):
    bounds = ([0,mean1-0.1*step, 0, 0,mean2-0.1*step, 0], 
              [0.2,mean1+0.1*step,(MAX-MIN)/8,1 ,mean2+0.1*step,(MAX-MIN)/8])
    popt, _ = curve_fit(gaussian, x, y, bounds = bounds)
    y_pred = [gaussian(X,popt[0],popt[1],popt[2],popt[3],popt[4],popt[5]) for X in x]
    y_true = y
    R2 = r2_score(y_true,y_pred)
    RMSQ = mean_squared_error(y_true,y_pred, squared=False)
    return R2,popt,RMSQ,bounds    

In [53]:
fig,gs = plt.figure(figsize=(54,16), dpi=300), GridSpec(1, 3)
ax1,ax2,ax3 = plt.subplot(gs[0:1,0:1]),plt.subplot(gs[0:1,1:2]),plt.subplot(gs[0:1,2:3])

label_size = 48
tick_size = 36
legend_size = 36


############# Fig One ################
_,__,___,____,_____,threshold = fitting_info[("O", "Al")].values()
x,y = length_2_distribution(OAl, threshold)


MIN = round(min(x),3)
MAX = round(max(x),3)
step = round(((MAX-MIN)/50),3)

np_mean1, np_mean2 = np.arange(MIN, MAX, step),np.arange(MIN, MAX, step)
dim = len(np_mean1)

np_R2, np_RMSQ = np.zeros([dim,dim]),np.zeros([dim,dim])
check_R2_list,check_mean_list = [],[]

for i, mean1 in enumerate(np_mean1):
    for j, mean2 in enumerate(np_mean2):
        try:
            R2, popt, RMSQ, bounds = test_r2(x, y, mean1, mean2, step, MIN, MAX)
            R2 = 0 if R2 < 0 else R2
            np_R2[i][j] = R2
            np_R2[j][i] = R2
            np_RMSQ[i][j] = RMSQ
            np_RMSQ[j][i] = RMSQ
            check_R2_list.append(R2)
            check_mean_list.append([mean1, mean2])
        except Exception as e:
            print(f"An error occurred: {e}")
            np_R2[i][j] = 0
            np_R2[j][i] = 0
            np_RMSQ[i][j] = max(np_RMSQ.flat)
            np_RMSQ[j][i] = max(np_RMSQ.flat)        
        if i == j:
            np_R2[i][j] = R2
            np_RMSQ[i][j] = RMSQ
            check_R2_list.append(R2)
            check_mean_list.append([mean1, mean2])


best_mean_combo = [j for i,j in zip(check_R2_list, check_mean_list) if i == max(check_R2_list)][0]
mean1, mean2 = best_mean_combo[0], best_mean_combo[1]
R2,popt,RMSQ,bounds = test_r2(x,y,mean1, mean2, step, MIN,MAX)
        

ax1.contourf(np_mean1, np_mean2, np_R2, 20, cmap="plasma")
ax1.set_xlabel("Mean_1 (Å)", fontsize=label_size, fontname='Times New Roman')
ax1.set_ylabel("Mean_2 (Å)", fontsize=label_size, fontname='Times New Roman')
ax1.tick_params(axis='x', labelsize=tick_size)
ax1.tick_params(axis='y', labelsize=tick_size)



############# Fig Two ################
_,__,___,____,_____,threshold = fitting_info[("O", "V")].values()
x,y = length_2_distribution(OV, threshold)


MIN = round(min(x),3)
MAX = round(max(x),3)
step = round(((MAX-MIN)/50),3)

np_mean1, np_mean2 = np.arange(MIN, MAX, step),np.arange(MIN, MAX, step)
dim = len(np_mean1)

np_R2, np_RMSQ = np.zeros([dim,dim]),np.zeros([dim,dim])
check_R2_list,check_mean_list = [],[]

for i, mean1 in enumerate(np_mean1):
    for j, mean2 in enumerate(np_mean2):
        try:
            R2, popt, RMSQ, bounds = test_r2(x, y, mean1, mean2, step, MIN, MAX)
            R2 = 0 if R2 < 0 else R2
            np_R2[i][j] = R2
            np_R2[j][i] = R2
            np_RMSQ[i][j] = RMSQ
            np_RMSQ[j][i] = RMSQ
            check_R2_list.append(R2)
            check_mean_list.append([mean1, mean2])
        except Exception as e:
            print(f"An error occurred: {e}")
            np_R2[i][j] = 0
            np_R2[j][i] = 0
            np_RMSQ[i][j] = max(np_RMSQ.flat)
            np_RMSQ[j][i] = max(np_RMSQ.flat)        
        if i == j:
            np_R2[i][j] = R2
            np_RMSQ[i][j] = RMSQ
            check_R2_list.append(R2)
            check_mean_list.append([mean1, mean2])


best_mean_combo = [j for i,j in zip(check_R2_list, check_mean_list) if i == max(check_R2_list)][0]
mean1, mean2 = best_mean_combo[0], best_mean_combo[1]
R2,popt,RMSQ,bounds = test_r2(x,y,mean1, mean2, step, MIN,MAX)
        

ax2.contourf(np_mean1, np_mean2, np_R2, 20, cmap="plasma")
ax2.set_xlabel("Mean_1 (Å)", fontsize=label_size, fontname='Times New Roman')
ax2.set_ylabel("Mean_2 (Å)", fontsize=label_size, fontname='Times New Roman')
ax2.tick_params(axis='x', labelsize=tick_size)
ax2.tick_params(axis='y', labelsize=tick_size)


############# Fig Three ################
plotting_R2_lists = sorted(list(map(lambda v:v["R2"], fitting_info_dict.values())))
x = [i for i in range(len(plotting_R2_lists))]
ax3.scatter(x, plotting_R2_lists, color='#0088FF', label='R2', alpha=1)

ax3.set_xlabel("Enumeration (#)", fontsize=label_size, fontname='Times New Roman')
ax3.set_ylabel("R2_score", fontsize=label_size, fontname='Times New Roman')
ax3.tick_params(axis='x', labelsize=tick_size)
ax3.tick_params(axis='y', labelsize=tick_size)
ax3.legend(loc='lower right', prop={'size': legend_size, 'family': 'Times New Roman'})


plt.subplots_adjust(left=0.05, right=0.98, bottom=0.08, top=0.95, wspace=0.25)
#fig.savefig("SI_Fig_2.svg")
plt.show()

In [75]:
import numpy as np
import plotly.graph_objects as go

x = np.linspace(0, np_R2.shape[1] - 1, np_R2.shape[1])
y = np.linspace(0, np_R2.shape[0] - 1, np_R2.shape[0])
X, Y = np.meshgrid(x, y)
fig = go.Figure()
fig.add_trace(go.Surface(z=np_R2, x=X, y=Y, colorscale='Blues', opacity=1))
fig.add_trace(go.Surface(z=np_RMSQ*30-1, x=X, y=Y, colorscale='reds', opacity=1))

fig.update_layout(
    title='3D Surface Plot with Two Surfaces',
    scene=dict(xaxis_title='X axis',yaxis_title='Y axis',
        zaxis_title='Z axis',
        zaxis=dict(range=[-1, 1])
    )
)

# Show the plot
fig.show()